In [3]:
import numpy as np
import fiona
import osr
import rasterio
import rasterio.features
from shapely.geometry import shape, mapping
from shapely.geometry.multipolygon import MultiPolygon


In [4]:
# Read input band with Rasterio
with rasterio.open('data/classified_result_195.tif') as src:
    src_band = src.read(1)
    # Keep track of unique pixel values in the input band
    unique_values = np.unique(src_band)
    # Polygonize with Rasterio. `shapes()` returns an iterable
    # of (geom, value) as tuples
    shapes = list(rasterio.features.shapes(src_band, transform=src.transform))

print('Loaded tif')

srs = osr.SpatialReference()      
srs.SetFromUserInput("EPSG:2232")  
crs = srs.ExportToProj4()        

print('Set up schema')
shp_schema = {
    'geometry': 'MultiPolygon',
    'properties': {'pixelvalue': 'int'}
}

print('Write out shapefile from tif')
# Convert tif to shapefile with pixel value as attribute field.
with fiona.open('data/classified_result_195_poly.shp', 'w', 'ESRI Shapefile', shp_schema, crs=crs) as shp:
    for pixel_value in unique_values:
        polygons = [shape(geom) for geom, value in shapes
                    if value == pixel_value]
        multipolygon = MultiPolygon(polygons)
        shp.write({
            'geometry': mapping(multipolygon),
            'properties': {'pixelvalue': int(pixel_value)}
        })
print('Done!')

Loaded tif
Set up schema
Write out shapefile from tif
Done!
